In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import sys
sys.path.insert(0, r'C:\Users\anass\Desktop\TUM\Courses\Practical\old_paper\shift-invariance-adv-robustness')

In [ ]:
import torch
import torch.nn as nn
import os
from advertorch.attacks import LinfPGDAttack, L2PGDAttack
#from models.basic_models import CNN, NeuralNet

#from utils import get_loaders_mnist
import argparse
import logging

In [ ]:
def get_loaders_mnist(classes, batch_size):
    # MNIST dataset
    train_dataset = CustomMNIST(root='../datasets/',
                                train=True,
                                transform=transforms.ToTensor(),
                                classes=classes)

    test_dataset = CustomMNIST(root='../datasets/',
                               train=False,
                               transform=transforms.ToTensor(),
                               classes=classes)

    # Data loader
    train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                               batch_size=batch_size,
                                               shuffle=False)

    test_loader = torch.utils.data.DataLoader(dataset=test_dataset,
                                              batch_size=1,
                                              shuffle=False)
    return train_loader, test_loader

In [ ]:
class CNN(nn.Module):
    def __init__(self, hidden_channels=1024, kernel_size=5, num_classes=1, no_final=False):
        super(CNN, self).__init__()
        self.hidden_channels = hidden_channels
        self.in_normalize = nn.BatchNorm2d(1)
        padding_size = kernel_size // 2
        self.conv1 = nn.Conv2d(1, hidden_channels, kernel_size=kernel_size,
                               padding=padding_size, padding_mode='circular')
        self.activation = nn.ReLU()
        self.pool = nn.AdaptiveAvgPool2d((1, 1))
        self.no_final = no_final
        self.final = nn.Linear(hidden_channels, num_classes)
        self.final_act = nn.Tanh()

    def forward(self, x):
        x = self.in_normalize(x)
        x = self.activation(self.conv1(x))
        x = self.pool(x)
        x = x.view(-1, self.hidden_channels * 1)
        x = self.final(x)
        if not self.no_final:
            x = self.final_act(x)
        return x.squeeze()


In [15]:
class NeuralNet(nn.Module):
    def __init__(self, input_size, hidden_layers, hidden_size, num_classes, no_final=False):
        super(NeuralNet, self).__init__()
        self.in_normalize = nn.BatchNorm1d(input_size)
        self.num_hidden_layers = hidden_layers
        self.input_size = input_size
        self.hidden_linear = nn.ModuleList()
        self.hidden_act = nn.ModuleList()
        for idx in range(hidden_layers):
            if idx == 0:
                self.hidden_linear.append(nn.Linear(input_size, hidden_size))
            else:
                self.hidden_linear.append(nn.Linear(hidden_size, hidden_size))
            self.hidden_act.append(nn.ReLU())

        self.final = nn.Linear(hidden_size, num_classes)
        self.no_final = no_final
        self.final_act = nn.Tanh()

    def forward(self, x):
        out = x.reshape(-1, self.input_size)
        out = self.in_normalize(out)
        for (linear, act) in zip(self.hidden_linear, self.hidden_act):
            out = linear(out)
            out = act(out)
        out = self.final(out)
        if not self.no_final:
            out = self.final_act(out)
        return out

In [16]:
logger = logging.getLogger(__name__)


In [17]:
def parse_args():
    parser = argparse.ArgumentParser()
    parser.add_argument('--model-type', choices=['cnn', 'fc'])
    parser.add_argument('--classes', type=str)
    parser.add_argument('--epochs', default=50, type=int)
    parser.add_argument('--batch-size', default=100, type=int)
    parser.add_argument('--hidden-size', default=5000, type=int)
    parser.add_argument('--hidden-layers', default=1, type=int)
    parser.add_argument('--kernel-size', default=7, type=int)
    parser.add_argument('--hidden-channels', default=1024, type=int)
    parser.add_argument('--learning-rate', default=0.001, type=float)
    parser.add_argument('--batches-use', default='all')
    parser.add_argument('--out-dir', type=str)
    parser.add_argument('--epsilons', type=str)
    parser.add_argument('--attack-type', choices=['l2', 'linf'])
    parser.add_argument('--eval-only', action='store_true')
    return parser.parse_args()

def train(model, train_loader, criterion, optimizer, device, num_epochs, batches_use):
    # Train the model
    total_step = len(train_loader)
    for epoch in range(num_epochs):
        correct = 0
        total = 0
        for i, (images, labels) in enumerate(train_loader):
            # Move tensors to the configured device
            #         images = images.reshape(-1, 28*28).to(device)
            images = images.to(device)
            labels = labels.to(device)

            # Forward pass
            outputs = model(images)
            loss = criterion(outputs, labels)

            total += labels.size(0)
            correct += (outputs.sign() == labels.sign()).sum().item()

            # Backprpagation and optimization
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

            if (i + 1) % 25 == 0 or (i + 1) % batches_use == 0:
                print('Epoch [{}/{}], Step [{}/{}], Loss: {:.4f}, Acc: {:.2f}'
                      .format(epoch + 1, num_epochs, i + 1, total_step, loss.item(), 100 * correct / total))
            if (i + 1) == batches_use:
                break
    logger.info("Total {} images used for training.".format(total))
    return model

def test(model, test_loader, device):
    # Test the model
    # In the test phase, don't need to compute gradients (for memory efficiency)
    with torch.no_grad():
        correct = 0
        total = 0
        for images, labels in test_loader:
            #         images = images.reshape(-1, 28*28).to(device)
            images = images.to(device)
            labels = labels.to(device)
            outputs = model(images)
            #         _, predicted = torch.max(outputs.data, 1)
            total += labels.size(0)
            correct += (outputs.sign() == labels.sign()).sum().item()

        logger.info('Accuracy of the network on the {} test images: {} %'.format(total, 100 * correct / total))

def test_ddn(model, test_loader, device):
    raise NotImplementedError
    return

def test_pgd(model, test_loader, device, epsilon, attack_type):
    # Accuracy counter
    correct = 0
    incorrect = 0
    adv_examples = []
    model = model.eval()

    if attack_type=='linf':
        adversary = LinfPGDAttack(model, loss_fn=nn.MSELoss(reduction="sum"), nb_iter=40, eps_iter=epsilon/20,
                                  rand_init=True, eps=epsilon, clip_min=0.0, clip_max=1.0, targeted = False)
    else:
        adversary = L2PGDAttack(model, loss_fn=nn.MSELoss(reduction="sum"), nb_iter=40, eps_iter=epsilon/20,
                                rand_init=True, eps=epsilon, clip_min=0.0, clip_max=1.0, targeted=False)

    # Loop over all examples in test set
    for data, target in test_loader:
        data, target = data.to(device), target.to(device)

        # Set requires_grad attribute of tensor. Important for Attack
        data.requires_grad = True

        # # Forward pass the data through the model
        #     output = model(data)
        # #         init_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        # # If the initial prediction is wrong, dont bother attacking, just move on
        #     if output.sign() != target.sign():
        #         incorrect+=1
        #         continue
        perturbed_data = adversary.perturb(data, target)
        #         # Re-classify the perturbed image
        new_output = model(perturbed_data)

        # Check for success
        #         final_pred = output.max(1, keepdim=True)[1] # get the index of the max log-probability
        #         if new_output.sign()==target.sign():
        if new_output.sign() == target.sign():
            correct += 1
        else:
            incorrect += 1


    # Calculate final accuracy for this epsilon
    final_acc = correct / float(correct + incorrect)
    logger.info("Attack Type: {}, Epsilon: {}\tTest Accuracy = {} / {} = {:.2f}".format(attack_type, epsilon,
                                                                                        correct, correct + incorrect,
                                                                                        100.*final_acc))

    # Return the accuracy and an adversarial example
    return final_acc, adv_examples


In [26]:
import os
os.getcwd()


'C:\\Users\\anass\\Desktop\\TUM\\Courses\\Practical\\old_paper\\shift-invariance-adv-robustness'

In [45]:
def parse_args():
    # Create an empty Namespace object to store the arguments
    args = argparse.Namespace()

    # Set the arguments manually
    args.model_type = 'cnn'
    args.classes = '0, 8'
    args.epochs = 50
    args.batch_size = 100
    args.hidden_size = 5000
    args.hidden_layers = 1
    args.kernel_size = 7
    args.hidden_channels = 1024
    args.learning_rate = 0.001
    args.batches_use = 'all'
    args.out_dir = os.getcwd()
    args.epsilons = '0.01, 0.02, 0.03, 0.04, 0.05, 0.06, 0.07, 0.08, 0.2, 0.3, 0.4, 0.5, 0.6, 0.7, 0.8'
    args.attack_type = 'l2'
    args.eval_only = False

    return args

In [46]:
args = parse_args()

In [47]:
# Check Device configuration
os.makedirs(args.out_dir, exist_ok=True)
logfile = os.path.join(args.out_dir, 'output.log')
logging.basicConfig(
    format='[%(asctime)s] - %(message)s',
    datefmt='%Y/%m/%d %H:%M:%S',
    level=logging.INFO,
    filename=logfile)
if not args.eval_only:
    logger.info(args)

In [48]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')


In [49]:

# Define Hyper-parameters
input_size = 784
hidden_size = args.hidden_size
hidden_layers = args.hidden_layers
classes = [int(x) for x in args.classes.split(',')]
hidden_channels = args.hidden_channels
kernel_size = args.kernel_size
num_classes = len(classes) - 1
num_epochs = args.epochs
batch_size = args.batch_size
learning_rate = args.learning_rate



In [50]:
import torch.nn as nn
import torch.nn.init as init
import torch
from torchvision.datasets import CIFAR10, FashionMNIST, MNIST, VisionDataset, SVHN
from torchvision import transforms
from PIL import Image
from torch.utils.data import random_split


In [51]:

class CustomMNIST(VisionDataset):
    training_file = 'training.pt'
    testing_file = 'test.pt'
    classes = ['0 - zero', '1 - one', '2 - two', '3 - three', '4 - four',
               '5 - five', '6 - six', '7 - seven', '8 - eight', '9 - nine']

    def __init__(self, root, train=True, transform=None, target_transform=None, classes=[0, 1]):
        super(CustomMNIST, self).__init__(root, transform=transform,
                                          target_transform=target_transform)
        assert len(classes) == 2, "Code was only implemented for 2 class MNIST, will need modifications"
        self.train = train
        if self.train:
            data_file = self.training_file
        else:
            data_file = self.testing_file
        self.data, self.targets = torch.load(os.path.join(self.processed_folder, data_file))
        select_indexes = [i for i, x in enumerate(self.targets.tolist()) if x in classes]
        #         targets = [-1 for i in select_indexes if self.targets[i]==classes[0] else 1]
        self.data, self.targets = self.data[select_indexes], self.targets[select_indexes]
        self.targets[self.targets == classes[0]] = 1.0
        self.targets[self.targets == classes[1]] = -1.0
        self.targets = self.targets.float()

    #         self.targets = sel

    def __getitem__(self, index):
        img, target = self.data[index], self.targets[index]
        img = Image.fromarray(img.numpy(), mode='L')
        if self.transform is not None:
            img = self.transform(img)

        if self.target_transform is not None:
            target = self.target_transform(target)

        return img, target

    def __len__(self):
        return len(self.data)

    @property
    def raw_folder(self):
        return os.path.join(self.root, 'MNIST', 'raw')

    @property
    def processed_folder(self):
        return os.path.join(self.root, 'MNIST', 'processed')

    @property
    def class_to_idx(self):
        return {_class: i for i, _class in enumerate(self.classes)}

    def _check_exists(self):
        return (os.path.exists(os.path.join(self.processed_folder,
                                            self.training_file)) and
                os.path.exists(os.path.join(self.processed_folder,
                                            self.test_file)))


In [ ]:
train_loader, test_loader = get_loaders_mnist(classes, batch_size)

In [ ]:

batches_use = int(args.batches_use) if args.batches_use!='all' else len(train_loader)

if args.model_type=='cnn':
    model = CNN(hidden_channels=hidden_channels, kernel_size=kernel_size, num_classes=num_classes)
else:
    model = NeuralNet(input_size, hidden_layers, hidden_size, num_classes).to(device)
model = model.to(device)
criterion = nn.MSELoss()
optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)
if not args.eval_only:
    model = train(model, train_loader, criterion, optimizer, device, num_epochs, batches_use)
    torch.save(model.state_dict(), os.path.join(args.out_dir, 'final_checkpoint.pt'))
    test(model, test_loader, device)
else:
    model.load_state_dict(torch.load(os.path.join(args.out_dir, 'final_checkpoint.pt')))

if args.ddn_only:
    test_ddn(model, test_loader, device)

epsilons = [float(x) for x in args.epsilons.split(',')]
for epsilon in epsilons:
    test_pgd(model, test_loader, device, epsilon, args.attack_type)